In [ ]:
# import des bibliothèques requises
import os
import openai
import requests

# données d'identification et d'authentification openAI
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")




# !!! SI LA LIGNE SUIVANTE EST EXECUTEE, SUPPRIMER LES OUTPUTS AVANT TOUT PUSH VERS GITHUB !!!

#print("Org successfully retrieved: ",openai.organization,"\nKey successfully retrieved: ",openai.api_key)

In [8]:
# le moteur IA utilisé dans ce programme
engine = "text-davinci-002"

# la liste suivante est ici hardcodée mais sera générée depuis une liste d'ingrédients suggérés par l'interface utilisateur (frontend)
ingredients_from_UI = ["beef","ginger","bell peppers","oregano"]

# à partir de la liste d'ingrédients, génération d'une chaîne de caractères qui s'intègrera naturellement dans la question posée
formatted_ingredients = "\n- " + "\n- ".join(ingredients_from_UI)

# la question qui sera posée au modèle openAI
question_asked_to_openAI_completions = "Provide a cooking recipe based on the following ingredients:\n" + formatted_ingredients + "\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions."

# un dictionnaire contenant les arguments à passer à l'endpoint "completions" via la méthode "openai.Completion.create" (SDK openAI)
completion_arguments = {
  "engine" : engine,
  "prompt" : question_asked_to_openAI_completions,
  "temperature" : 0.7,
  "top_p" : 1,
  "max_tokens" : 300,
  "frequency_penalty" : 0,
  "presence_penalty" : 0
  }

print(completion_arguments)


{'engine': 'text-davinci-002', 'prompt': "Provide a cooking recipe based on the following ingredients:\n\n- beef\n- ginger\n- bell peppers\n- oregano\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions.", 'temperature': 0.7, 'top_p': 1, 'max_tokens': 300, 'frequency_penalty': 0, 'presence_penalty': 0}


In [9]:
# requête openAI paramétrée

response = openai.Completion.create(
  engine = completion_arguments["engine"],
  prompt =  completion_arguments["prompt"],
  temperature = completion_arguments["temperature"],
  top_p = completion_arguments["top_p"],
  max_tokens = completion_arguments["max_tokens"],
  frequency_penalty = completion_arguments["frequency_penalty"],
  presence_penalty = completion_arguments["presence_penalty"])


In [10]:
complete_recipe = response["choices"][0]["text"][2:]

recipe_name = complete_recipe[21:complete_recipe.find("\n")-1]

ingredients_extraction = complete_recipe[complete_recipe.find("Ingredients:") + 14 : complete_recipe.find("Instructions:")-2]
instructions_extraction = complete_recipe[complete_recipe.find("Instructions:") + 15:]

ingredients_list = list(ingredients_extraction.split('\n'))
ingredients_list = list(i.strip('- ') for i in ingredients_list)
ingredients_list = list(filter(lambda a: a != '', ingredients_list))

instructions_list = list(instructions_extraction.split('\n'))
instructions_list = list(i[3:] for i in instructions_list)
instructions_list = list(filter(lambda a: a != '', instructions_list))

print("Nom de la recette : \n" + recipe_name)
print("Liste d'ingrédients : \n" + "\n".join(ingredients_list))
print("Recette : \n" + "\n".join(instructions_list))

Nom de la recette : 
Ginger and Bell Pepper
Liste d'ingrédients : 
1 pound beef, thinly sliced
1 tablespoon ginger, minced
2 bell peppers, thinly sliced
1 teaspoon oregano
1 tablespoon olive oil
1/4 cup soy sauce
1/4 cup beef broth
salt and pepper to taste
Recette : 
eason the beef with salt and pepper.
In a large skillet or wok, heat the oil over high heat.
Add the beef and cook until browned.
Add the ginger, bell peppers, and oregano, and cook for 1-2 minutes.
Add the soy sauce and beef broth, and cook until the sauce has thickened.
Serve with rice or noodles.


In [17]:
# définition de la fonction qui trouvera une recette en fonction d'ingrédients fournis
# la fonction ne prend pour l'instant qu'un seul argument

def findrecipe(ingredients, temperature):
    response = openai.Completion.create(
        engine = completion_arguments["engine"],
        prompt =  completion_arguments["prompt"],
        temperature = temperature,
        top_p = completion_arguments["top_p"],
        max_tokens = completion_arguments["max_tokens"],
        frequency_penalty = completion_arguments["frequency_penalty"],
        presence_penalty = completion_arguments["presence_penalty"])

    complete_recipe = response["choices"][0]["text"][2:]

    recipe_name = complete_recipe[:complete_recipe.find("\n")-1]

    ingredients_extraction = complete_recipe[complete_recipe.find("Ingredients:") + 14 : complete_recipe.find("Instructions:")-2]
    instructions_extraction = complete_recipe[complete_recipe.find("Instructions:") + 15:]

    ingredients_list = list(ingredients_extraction.split('\n'))
    ingredients_list = list(i.strip('- ') for i in ingredients_list)
    ingredients_list = list(filter(lambda a: a != '', ingredients_list))

    instructions_list = list(instructions_extraction.split('\n'))
    instructions_list = list(i[3:] for i in instructions_list)
    instructions_list = list(filter(lambda a: a != '', instructions_list))

    print("=========================================\n"+str(response)+"\n=========================================\n")

    return {
        "RecipeName" : recipe_name,
        "Ingredients" : ingredients_list,
        "Instructions" : instructions_list
    }


In [18]:
# appel de la fonction

findrecipe(ingredients_from_UI,.7)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nPepper Steak\n\n- 1 pound beef\n- 1 teaspoon ginger\n- 1 bell pepper\n- 1 teaspoon oregano\n\nInstructions:\n\n1. Cut the beef into thin strips.\n2. Peel and chop the ginger.\n3. Cut the bell pepper into thin strips.\n4. Mix the beef, ginger, bell pepper, and oregano together in a bowl.\n5. Heat a pan over medium-high heat and cook the beef mixture until the beef is cooked through.\n6. Serve with rice."
    }
  ],
  "created": 1666585088,
  "id": "cmpl-64jCaIMd9U9IW38m32g4YIOE0OjvX",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 116,
    "prompt_tokens": 49,
    "total_tokens": 165
  }
}



{'RecipeName': 'Pepper Stea',
 'Ingredients': ['1 pound beef',
  '1 teaspoon ginger',
  '1 bell pepper',
  '1 teaspoon oregano'],
 'Instructions': ['Cut the beef into thin strips.',
  'Peel and chop the ginger.',
  'Cut the bell pepper into thin strips.',
  'Mix the beef, ginger, bell pepper, and oregano together in a bowl.',
  'Heat a pan over medium-high heat and cook the beef mixture until the beef is cooked through.',
  'Serve with rice.']}